In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import SGD, Adam
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
train_path = r"data/sign_mnist_train.csv"
test_path = r"data/sign_mnist_test.csv"

In [10]:
class SMDataset(Dataset):
    def __init__(self, file_path):
        data = pd.read_csv(file_path)
        self.y = data["label"]
        self.y = F.one_hot(torch.tensor(self.y.values)).float()
        del data["label"]
        self.x = (torch.tensor(data.values) / 255)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]   

In [11]:
train_data = SMDataset(train_path)
test_data = SMDataset(test_path)

In [12]:
X_train, y_train = train_data[:]
X_test, y_test = test_data[:]

In [13]:
display(X_train.dtype, y_train.dtype)

torch.float32

torch.float32

In [14]:
display(X_train.shape, y_train.shape)

torch.Size([27455, 784])

torch.Size([27455, 25])

In [28]:
class SMNN(nn.Module):
    def __init__(self, weight_decay=0):
        super().__init__()
        self.layer1 = nn.Linear(784, 100)
        self.layer2 = nn.Linear(100, 50)
        self.layer3 = nn.Linear(50, 25)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=weight_decay)
    def forward(self, x):
#         x = x.view(-1, 784)
        x = self.dropout(self.relu(self.layer1(x)))
        x = self.dropout(self.relu(self.layer2(x)))
        x = self.dropout(self.layer3(x))
        return x

In [32]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(784, 100)
        self.act = nn.ReLU()
        self.output = nn.Linear(100, 25)
        self.prob = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x

In [29]:
model_fc = SMNN()

In [36]:
model = NeuralNetClassifier(PimaClassifier,
                            criterion=nn.BCELoss,
                            max_epochs=100,
                            batch_size=10,
                            verbose=False
)
param_grid = {'optimizer': [optim.SGD, optim.RMSprop, optim.Adagrad, optim.Adadelta, 
                            optim.Adam, optim.Adamax, optim.NAdam],
}

In [43]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, error_score='raise')
grid_result = grid.fit(X_train, y_train)

ValueError: Stratified CV requires explicitly passing a suitable y.